# Monk Notebook

In [ ]:
from __future__ import annotations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from model.network import NeuralNetwork
from model.trainer import Trainer
from model.losses import Loss
from utils import DataLoader
from utils import load_monk, plot_curves
from utils.model_selection_helpers import count_parameters
from model.activations import sigmoid
from model.losses import mse
from utils.grid_search import grid_search_monk
from IPython.display import clear_output

np.random.seed(8) #reproducibility

In [ ]:
# All the configurations to test for the training
# Neural Network architectures
INPUT_NEURONS = 17
OUTPUT_NEURONS = 1

HIDDEN_LAYER_SIZES = [4, 8]
HIDDEN_LAYERS_COUNTS = [1, 2]
INTERNAL_ACTIVATIONS = ['tanh', 'leaky relu', 'relu']
OUTPUT_ACTIVATIONS_AND_LOSS = [('sigmoid', 'mse'), ('identity', 'binary cross entropy sigmoid')]
NEURAL_NETWORK_CONFIGURATIONS = []

for hidden_layers_count in HIDDEN_LAYERS_COUNTS:
    for hidden_layer_size in HIDDEN_LAYER_SIZES:
        for internal_activation in INTERNAL_ACTIVATIONS:
            for output_activation, loss_function in OUTPUT_ACTIVATIONS_AND_LOSS:
                architecture = [INPUT_NEURONS] + [hidden_layer_size] * hidden_layers_count + [OUTPUT_NEURONS]
                activations = [internal_activation] * hidden_layers_count + [output_activation]
                NEURAL_NETWORK_CONFIGURATIONS.append((architecture, activations, loss_function))

# Training parameters
ETA_CONFIGURATIONS = [0.25, 0.1]
LAMBDA_CONFIGURATIONS = [0, 1e-1, 1e-2, 1e-3]
ALPHA_CONFIGURATIONS = [0, 0.5, 0.9]
BATCH_SIZES = [32, -1]

# Cross-validation parameters
K_FOLDS= 5
EPOCHS = 500
EARLY_STOPPING_PATIENCE = 50

# All possible configurations are tuples (NEURAL_NETWORK_ARCHITECTURE, NEURAL_NETWORK_ACTIVATION, LOSS_F, ETA, LAMBDA, ALPHA, BATCH_SIZE)
CONFIGURATIONS = []

for NEURAL_NETWORK_ARCHITECTURE, NEURAL_NETWORK_ACTIVATION, LOSS_F in NEURAL_NETWORK_CONFIGURATIONS:
  for ETA in ETA_CONFIGURATIONS:
    for LAMBDA in LAMBDA_CONFIGURATIONS:
      for ALPHA in ALPHA_CONFIGURATIONS:
        for BATCH_SIZE in BATCH_SIZES:
            config = (NEURAL_NETWORK_ARCHITECTURE, NEURAL_NETWORK_ACTIVATION, LOSS_F, ETA, LAMBDA, ALPHA, BATCH_SIZE)
            CONFIGURATIONS.append(config)
LEN_CONFIGURATIONS = len(CONFIGURATIONS)

print(f"Total configurations: {LEN_CONFIGURATIONS}")

In [ ]:
# CONFIG_DICTIONARY is the id of each configuration of CONFIGURATIONS
# CONFIG_DICTIONARY_INSTABILITY_VAL is the sumatory of the relative value of when val loss raises
# CONFIG_DICTIONARY_INSTABILITY_TRAIN is the sumatory of the relative value of when train loss raises
# CONFIG_DICTIONARY_EPOCHS stores the epochs needed for each configuration to train
# CONFIG_DICTIONARY_TRAIN_LOSS_DIFF stores the sumatory of when train loss < val loss, so the model is overfitting

def print_top_25(CONFIG_DICTIONARY, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF, min_acc = 100):
    # print the top 25 configurations, sorting criterias: 1 avg accuracy (no longer valued if > than min_acc), 2 n parameters (the less the better), 3 val instability coeff (the less the better)
    TOP_25_CONFIGS_INDEXES = sorted(CONFIG_DICTIONARY, key=lambda i: (-min(min_acc, CONFIG_DICTIONARY[i]), count_parameters(CONFIGURATIONS[i][0]), (CONFIG_DICTIONARY_INSTABILITY_VAL[i]) / K_FOLDS))[:25]
    print("Top 5 configurations:")
    for i in TOP_25_CONFIGS_INDEXES:
        ACCURACY = CONFIG_DICTIONARY[i] * 100 / K_FOLDS
        print(f'''Config index: {CONFIGURATIONS[i]}, Mean Accuracy: {ACCURACY}%,
            training instability coeff validation: {CONFIG_DICTIONARY_INSTABILITY_VAL[i] / K_FOLDS}, 
            training instability coeff train: {CONFIG_DICTIONARY_INSTABILITY_TRAIN[i] / K_FOLDS}, 
            training loss-val loss diff: {CONFIG_DICTIONARY_TRAIN_LOSS_DIFF[i] / K_FOLDS}, 
            Mean Epochs: {CONFIG_DICTIONARY_EPOCHS[i] // K_FOLDS}''')

## Monk 1

In [ ]:
# Here we load the dataset and create k folds
PATH_TRAIN = 'data/monk/monks-1.train'
PATH_TEST = 'data/monk/monks-1.test'
X_train_full, y_train_full, X_test, y_test = load_monk(PATH_TRAIN, PATH_TEST)
monk_dataset_1 = DataLoader(X_train_full, y_train_full)
k_fold = monk_dataset_1.k_fold(k = K_FOLDS)

In [ ]:
grid_s_outputs = grid_search_monk(LEN_CONFIGURATIONS, CONFIGURATIONS, k_fold, EPOCHS, EARLY_STOPPING_PATIENCE)
clear_output(wait=False)
CONFIG_DICTIONARY, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF = grid_s_outputs
print_top_25(CONFIG_DICTIONARY, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF)

## Monk 2

In [ ]:
# Here we load the dataset and create k folds
PATH_TRAIN = 'data/monk/monks-2.train'
PATH_TEST = 'data/monk/monks-2.test'
X_train_full, y_train_full, X_test, y_test = load_monk(PATH_TRAIN, PATH_TEST)
monk_dataset_2 = DataLoader(X_train_full, y_train_full)
k_fold = monk_dataset_2.k_fold(k = K_FOLDS)

In [ ]:
grid_s_outputs = grid_search_monk(LEN_CONFIGURATIONS, CONFIGURATIONS, k_fold, EPOCHS = 500, EARLY_STOPPING_PATIENCE = EARLY_STOPPING_PATIENCE)
clear_output(wait=False)
CONFIG_DICTIONARY, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF = grid_s_outputs
print_top_25(CONFIG_DICTIONARY, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF)

## Monk 3

In [ ]:
# Here we load the dataset and create k folds
PATH_TRAIN = 'data/monk/monks-3.train'
PATH_TEST = 'data/monk/monks-3.test'
X_train_full, y_train_full, X_test, y_test = load_monk(PATH_TRAIN, PATH_TEST)
monk_dataset_3 = DataLoader(X_train_full, y_train_full)
k_fold = monk_dataset_3.k_fold(k = K_FOLDS)

In [ ]:
grid_s_outputs = grid_search_monk(LEN_CONFIGURATIONS, CONFIGURATIONS, k_fold, EPOCHS = 500, EARLY_STOPPING_PATIENCE = EARLY_STOPPING_PATIENCE)
clear_output(wait=False)
CONFIG_DICTIONARY, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF = grid_s_outputs
print_top_25(CONFIG_DICTIONARY, CONFIG_DICTIONARY_INSTABILITY_VAL, CONFIG_DICTIONARY_INSTABILITY_TRAIN, CONFIG_DICTIONARY_EPOCHS, CONFIG_DICTIONARY_TRAIN_LOSS_DIFF, min_acc=93.5)

## Monk 3 no reg

In [ ]:
#lets choose an overfitting config here
# print the top 25 configurations, sorting criterias: 1 avg accuracy (no longer valued if > than min_acc), 2 n parameters (the less the better), 3 val instability coeff (the less the better)
TOP_25_CONFIGS_INDEXES = sorted(CONFIG_DICTIONARY, key=lambda i: (-count_parameters(CONFIGURATIONS[i][0]), -CONFIG_DICTIONARY[i], (CONFIG_DICTIONARY_INSTABILITY_VAL[i]) / K_FOLDS))[:100]
print("Top 5 configurations:")
for i in TOP_25_CONFIGS_INDEXES:
    ACCURACY = CONFIG_DICTIONARY[i] * 100 / K_FOLDS
    print(f'''Config index: {CONFIGURATIONS[i]}, Mean Accuracy: {ACCURACY}%,
        training instability coeff validation: {CONFIG_DICTIONARY_INSTABILITY_VAL[i] / K_FOLDS}, 
        training instability coeff train: {CONFIG_DICTIONARY_INSTABILITY_TRAIN[i] / K_FOLDS}, 
        training loss-val loss diff: {CONFIG_DICTIONARY_TRAIN_LOSS_DIFF[i] / K_FOLDS}, 
        Mean Epochs: {CONFIG_DICTIONARY_EPOCHS[i] // K_FOLDS}''')